##Question Classification:

In [1]:
#importing Necessary Libraries
import torch
import random
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW

# Set a random seed for reproducibility
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)

# Load the pre-trained model and tokenizer
model_name = "distilbert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define the training and validation datasets
train_texts = [
    "What department handles sales?",
    "What is the role of an employee in HR?",
    "Any news about the company?",
    "What are the requirements for a software engineer position?",
    "How does the marketing department contribute to the company's growth?",
    "Any updates on the company's recent product launch?",
    "What are the benefits provided to employees?",
    "How can I apply for a job in the finance department?",
    "Are there any recent changes in the company's leadership?",
    "What is the typical work schedule for a customer support representative?"
]
train_labels = [0, 1, 2, 1, 0, 2, 1, 0, 2, 1]  # Corresponding labels for each training example

val_texts = [
    "How can I contact the IT department?",
    "What are the responsibilities of a manager?",
    "Any updates on the company's financial performance?",
    "What qualifications are required for a sales representative position?",
    "How does the human resources department handle employee benefits?",
    "Are there any recent press releases from the company?",
    "What is the process for requesting time off?",
    "How can I reach the customer service department?",
    "Any updates on the company's expansion plans?",
    "What is the average salary for a software developer?"
]
val_labels = [0, 1, 2, 1, 0, 2, 1, 0, 2, 1]  # Corresponding labels for each validation example

# Tokenize the texts and create input tensors
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

train_dataset = torch.utils.data.TensorDataset(
    torch.tensor(train_encodings["input_ids"]),
    torch.tensor(train_encodings["attention_mask"]),
    torch.tensor(train_labels)
)
val_dataset = torch.utils.data.TensorDataset(
    torch.tensor(val_encodings["input_ids"]),
    torch.tensor(val_encodings["attention_mask"]),
    torch.tensor(val_labels)
)

# Training hyperparameters
batch_size = 16
learning_rate = 2e-5
num_epochs = 3

# Optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=learning_rate)

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training loop
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

for epoch in range(num_epochs):
    model.train()
    for batch in train_dataloader:
        input_ids, attention_mask, labels = batch
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    val_loss = 0
    correct_predictions = 0
    total_predictions = 0

    with torch.no_grad():
        for batch in val_dataloader:
            input_ids, attention_mask, labels = batch
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            logits = outputs.logits

            val_loss += loss.item()

            predicted_labels = logits.argmax(dim=1)
            correct_predictions += (predicted_labels == labels).sum().item()
            total_predictions += len(labels)

    avg_val_loss = val_loss / len(val_dataloader)
    val_accuracy = correct_predictions / total_predictions

    print(f"Epoch {epoch + 1} - Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

# Save the trained model
model.save_pretrained("./question_classification_model")
tokenizer.save_pretrained("./question_classification_model")

# Example usage after training
def classify_question(question):
    # Load the fine-tuned model and tokenizer
    model = AutoModelForSequenceClassification.from_pretrained("./question_classification_model")
    tokenizer = AutoTokenizer.from_pretrained("./question_classification_model")

    # Tokenize the question
    inputs = tokenizer.encode_plus(
        question,
        add_special_tokens=True,
        return_tensors="pt"
    )

    # Classify the question
    inputs = {k: v.to(device) for k, v in inputs.items()}
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = logits.argmax().item()

    # Define the categories
    categories = ["Company Department", "Employee Role", "Company News"]

    # Return the predicted category
    return categories[predicted_class]

# Example usage
question = "What department handles sales?"
predicted_category = classify_question(question)
print(f"Predicted category: {predicted_category}")


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'classifi

Epoch 1 - Validation Loss: 1.0792, Validation Accuracy: 0.9000
Epoch 2 - Validation Loss: 1.0565, Validation Accuracy: 0.9000
Epoch 3 - Validation Loss: 1.0328, Validation Accuracy: 0.7000
Predicted category: Employee Role


#Information Retrieval:

In [2]:
import sqlite3

# Create the SQLite database
conn = sqlite3.connect("employee_database.db")
db_cursor = conn.cursor()

# Drop the employees table if it already exists
db_cursor.execute("DROP TABLE IF EXISTS employees")

# Create the employees table
db_cursor.execute("""
    CREATE TABLE employees (
        id INTEGER PRIMARY KEY,
        name TEXT,
        role TEXT,
        department TEXT
    )
""")

# Insert sample employee data
employees_data = [
    (1, "John Doe", "sales representative", "Sales"),
    (2, "Jane Smith", "customer support representative", "Customer Support"),
    (3, "Michael Johnson", "software engineer", "Engineering"),
    (4, "Emily Davis", "marketing specialist", "Marketing"),
    (5, "Robert Brown", "HR manager", "HR")
]

db_cursor.executemany("INSERT INTO employees VALUES (?, ?, ?, ?)", employees_data)

# Commit the changes and close the database connection
conn.commit()
db_cursor.close()
conn.close()


In [3]:
import csv

# Define the employee details
employee_details = [
    {"id": 1, "name": "John Doe", "role": "sales representative", "department": "Sales"},
    {"id": 2, "name": "Jane Smith", "role": "customer support representative", "department": "Customer Support"},
    {"id": 3, "name": "Michael Johnson", "role": "software engineer", "department": "Engineering"},
    {"id": 4, "name": "Emily Davis", "role": "marketing specialist", "department": "Marketing"},
    {"id": 5, "name": "Robert Brown", "role": "HR manager", "department": "HR"}
]

# Define the field names for the CSV file
field_names = ["id", "name", "role", "department"]

# Write the employee details to the CSV file
with open("employee_details.csv", "w", newline="") as file:
    writer = csv.DictWriter(file, fieldnames=field_names)
    writer.writeheader()
    writer.writerows(employee_details)


In [4]:
import csv
import requests
import sqlite3

# Function to retrieve employee details from an SQL database
def retrieve_employee_details(employee_role):
    # Connect to the SQL database
    conn = sqlite3.connect("employee_database.db")
    cursor = conn.cursor()

    # Execute the SQL query to retrieve employee details
    cursor.execute("SELECT * FROM employees WHERE role=?", (employee_role,))
    rows = cursor.fetchall()

    # Close the database connection
    cursor.close()
    conn.close()

    # Return the retrieved employee details
    return rows

# Function to retrieve company news from a news API
def retrieve_company_news():
    # Make a request to the news API
    response = requests.get("https://api.example.com/news")

    # Parse the response and retrieve the company news
    news_data = response.json()
    company_news = news_data["company_news"]

    # Return the retrieved company news
    return company_news

# Function to retrieve department details from a CSV file
def retrieve_department_details(department):
    # Read the CSV file
    with open("department_details.csv", "r") as file:
        reader = csv.DictReader(file)
        rows = [row for row in reader if row["department"] == department]

    # Return the retrieved department details
    return rows

# Example usage
question = "What department handles sales?"

# Classify the question using the question classification model
predicted_category = classify_question(question)

# Retrieve relevant information based on the predicted category
if predicted_category == "Company Department":
    department_details = retrieve_department_details("sales")
    print("Department Details:")
    for row in department_details:
        print(row["name"], row["description"])
elif predicted_category == "Employee Role":
    employee_role = "sales representative"
    employee_details = retrieve_employee_details(employee_role)
    print("Employee Details:")
    for row in employee_details:
        print(row[0], row[1], row[2])
elif predicted_category == "Company News":
    company_news = retrieve_company_news()
    print("Company News:")
    for news in company_news:
        print(news["title"], news["description"])
else:
    print("Invalid category")


Employee Details:
1 John Doe sales representative


#Answer Generation:

In [5]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
model = AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

# User's question
question = "What is the sales department responsible for?"

# Retrieved information as context
context = "The sales department is responsible for handling the sales operations of the company. They manage client relationships, negotiate deals, and meet sales targets."

# Tokenize the input
inputs = tokenizer.encode_plus(question, context, add_special_tokens=True, return_tensors="pt")

# Generate potential answer
answer_start_scores, answer_end_scores = model(inputs["input_ids"], attention_mask=inputs["attention_mask"]).values()
answer_start = torch.argmax(answer_start_scores)
answer_end = torch.argmax(answer_end_scores) + 1

# Decode and print the potential answer
answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end]))
print("Potential Answer:", answer)


Potential Answer: sales operations of the company


# Answer Validation:

In [6]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import re

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
model = AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

# User's question
question = "What is the sales department responsible for?"

# Retrieved information as context
context = "The sales department is responsible for handling the sales operations of the company. They manage client relationships, negotiate deals, and meet sales targets."

# Tokenize the input
inputs = tokenizer.encode_plus(question, context, add_special_tokens=True, return_tensors="pt")

# Generate potential answer
answer_start_scores, answer_end_scores = model(inputs["input_ids"], attention_mask=inputs["attention_mask"]).values()
answer_start = torch.argmax(answer_start_scores)
answer_end = torch.argmax(answer_end_scores) + 1

# Decode the potential answer
potential_answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end]))

# Validate the answer
# Check if the potential answer is present in the context
if potential_answer.lower() in context.lower():
    print("Answer is present in the context.")
else:
    print("Answer is not present in the context.")

# Check if the potential answer contains keywords related to the question
keywords = re.findall(r'\b\w+\b', question.lower())
if any(keyword.lower() in potential_answer.lower() for keyword in keywords):
    print("Answer contains keywords related to the question.")
else:
    print("Answer does not contain keywords related to the question.")


Answer is present in the context.
Answer contains keywords related to the question.


#API Structure: 

In [7]:
from flask import Flask, request, jsonify
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import re

app = Flask(__name__)

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
model = AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

@app.route('/api/answer', methods=['POST'])
def get_answer():
    # Get the question from the request
    question = request.json['question']

    # Get the context from the request
    context = request.json['context']

    # Tokenize the input
    inputs = tokenizer.encode_plus(question, context, add_special_tokens=True, return_tensors="pt")

    # Generate potential answer
    answer_start_scores, answer_end_scores = model(inputs["input_ids"], attention_mask=inputs["attention_mask"]).values()
    answer_start = torch.argmax(answer_start_scores)
    answer_end = torch.argmax(answer_end_scores) + 1

    # Decode the potential answer
    potential_answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end]))

    # Validate the answer
    # Check if the potential answer is present in the context
    answer_present = potential_answer.lower() in context.lower()

    # Check if the potential answer contains keywords related to the question
    keywords = re.findall(r'\b\w+\b', question.lower())
    answer_contains_keywords = any(keyword.lower() in potential_answer.lower() for keyword in keywords)

    # Prepare the response
    response = {
        'question': question,
        'context': context,
        'answer': potential_answer,
        'answer_present': answer_present,
        'answer_contains_keywords': answer_contains_keywords
    }

    return jsonify(response)

if __name__ == '__main__':
    app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [20/May/2023 20:03:18] "GET /api/answer HTTP/1.1" 405 -
127.0.0.1 - - [20/May/2023 20:03:18] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [20/May/2023 20:03:47] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [20/May/2023 20:03:47] "GET /favicon.ico HTTP/1.1" 404 -


In [ ]:
import requests

url = 'http://localhost:5000/api/answer'
payload = {
    'question': 'What is the sales department responsible for?',
    'context': 'The sales department is responsible for handling the sales operations of the company. They manage client relationships, negotiate deals, and meet sales targets.'
}

response = requests.post(url, json=payload)
print(response.json())
